In [1]:
!pip install flask-ngrok
!pip install -U -q kaggle

import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense, GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, load_model
from keras.optimizers import Adam
from google.colab import files
from flask import Flask
from flask_ngrok import run_with_ngrok

Using TensorFlow backend.


In [2]:
# Mounting our Google Drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
# Setting up kaggle
!mkdir  /root/.kaggle

files.upload()
!cp kaggle.json /root/.kaggle

Saving kaggle.json to kaggle.json


In [4]:
# Download dataset from Kaggle
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

100% 2.29G/2.29G [00:35<00:00, 56.9MB/s]
100% 2.29G/2.29G [00:35<00:00, 69.5MB/s]


In [5]:
# Preparing the dataset
!apt install pv
!unzip -o /content/chest-xray-pneumonia.zip  | pv -l >/dev/null

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
Suggested packages:
  doc-base
The following NEW packages will be installed:
  pv
0 upgraded, 1 newly installed, 0 to remove and 7 not upgraded.
Need to get 48.3 kB of archives.
After this operation, 123 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 pv amd64 1.6.6-1 [48.3 kB]
Fetched 48.3 kB in 0s (1,077 kB/s)
Selecting previously unselected package pv.
(Reading database ... 145674 files and directories currently installed.)
Preparing to unpack .../archives/pv_1.6.6-1_amd64.deb ...
Unpacking pv (1.6.6-1) ...
Setting up pv (1.6.6-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
17.6k 0:00:50 [ 351 /s] [                                               <=>    ]


In [6]:
# Simple model for training
base_model = MobileNet(weights='imagenet', include_top=False)  

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)  # dense layer 1
x = Dense(256, activation='relu')(x)  # dense layer 2
preds = Dense(2, activation='softmax')(x)  # final layer with softmax activation

model = Model(inputs=base_model.input, outputs=preds)

for layer in model.layers[:20]:
    layer.trainable = False
for layer in model.layers[20:]:
    layer.trainable = True

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '












17227776/17225924 [==============================] - 1s 0us/step


In [7]:
'''
Adam optimizer
Loss function will be categorical cross entropy
Evaluation metric will be accuracy
'''
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)  
train_generator = train_datagen.flow_from_directory('/content/chest_xray/train/',
                                                    target_size=(224, 224),
                                                    color_mode='rgb',
                                                    batch_size=32,
                                                    class_mode='categorical', shuffle=True)

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

step_size_train = train_generator.n // train_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    epochs=10)

Found 5216 images belonging to 2 classes.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
163/163 [==============================] - 69s 423ms/step - loss: 0.1000 - acc: 0.9618
Epoch 2/10
163/163 [==============================] - 60s 369ms/step - loss: 0.0479 - acc: 0.9826
Epoch 3/10
163/163 [==============================] - 60s 366ms/step - loss: 0.0371 - acc: 0.9881
Epoch 4/10
163/163 [==============================] - 60s 367ms/step - loss: 0.0319 - acc: 0.9889
Epoch 5/10
163/163 [==============================] - 60s 366ms/step - loss: 0.0236 - acc: 0.9923
Epoch 6/10
163/163 [==============================] - 60s 368ms/step - loss: 0.0233 - acc: 0.9921
Epoch 7/10
163/163 [==============================] - 59s 364ms/step - loss: 0.0161 - acc: 0.9939
Epoch 8/10
163/163 [==============================] - 60s 367ms/step - loss: 0.0192 - acc: 0.9954
Epoch 9/10
163/163 [==============================] - 60s 367ms/step - loss:

In [0]:
# Saving our model
model.save('chest-xray-pneumonia-mobilenet.h5')

In [0]:
# Uploading it to our Google Drive
!cp -r '/content/chest_xray/chest_xray' '/content/drive/My Drive/pneumonia'
!cp chest-xray-pneumonia-mobilenet.h5 '/content/drive/My Drive/pneumonia'

In [10]:
# Loading our model
new_model = load_model("chest-xray-pneumonia-mobilenet.h5")

# Testing our model
def predict_img(pic):
    img = image.load_img(pic, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    normal_score, pneumonia_score = np.around(new_model.predict(x), decimals=2)[0]
    return {'normal_score': normal_score, 'pneumonia_score': pneumonia_score}

# Path to images
ill_path = "/content/chest_xray/train/PNEUMONIA/"
good_path = "/content/chest_xray/train/NORMAL/"
ill_pic = ill_path + os.listdir(ill_path)[3]
good_pic = good_path + os.listdir(good_path)[3]

print(predict_img(ill_pic))
print(predict_img(good_pic))

{'normal_score': 0.0, 'pneumonia_score': 1.0}
{'normal_score': 1.0, 'pneumonia_score': 0.0}
